<hr style="height:3px;border:none;color:#333;background-color:#333;" />
<img style=" float:right; display:inline" src="http://opencloud.utsa.edu/wp-content/themes/utsa-oci/images/logo.png"/>

### **University of Texas at San Antonio** 
<br/>
<br/>
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 2.5em;"> **Open Cloud Institute** </span>

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

### Big Data Analysis with Spark

<br/>
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.5em;"> **Paul Rad, Ph.D.** </span>  


<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.5em;"> **Nimish Joshi, Research Fellow** </span>

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.4em;"> *Open Cloud Institute, University of Texas at San Antonio, San Antonio, Texas, USA* </span>  
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.4em;"> {Nimish.Joshi, Paul.Rad}@utsa.edu </span>  

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.5em;"> **Example: Pi.py:** </span>

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;"> Let's have a look on example *pi.py*: We first call few libraries as below. </span>


In [ ]:
#import sys
#from random import random
#from operator import add


<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;"> As we are aware of requirement of using SparkContext linrary too, which is also needs to import. </span>

In [ ]:
#from pyspark import SparkContext


<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;"> Now, in the main function, we will perform the operation for calulating pi as followed. Here we are using the system argument to calculate the pi value </span>

In [ ]:
"""sc = SparkContext(appName = "PythonPi")
partitions = int(sys.argv[1]) if len(sys.argv) > 1 else 2
n = 100000 * partitions
def(f):
    x = random()* 2 - 1
    y = random()* 2 - 1
    return 1 if x ** 2 + y ** 2 < 1 else 0
count = sc.parallelize(range(1, n+1), partitions).map(f).reduce(add)
print("Pi is roughly %f" % (4.0 * count / n))
sc.stop""""""

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;"> combining everything togather into the executable code: </span>

In [ ]:
"""
# Executable file

import sys
from random import random
from operator import add

from pyspark import SparkContext


if __name__ == "__main__":
    """
        Usage: pi [partitions]
    """
    sc = SparkContext(appName="PythonPi")
    partitions = int(sys.argv[1]) if len(sys.argv) > 1 else 2
    n = 100000 * partitions

    def f(_):
        x = random() * 2 - 1
        y = random() * 2 - 1
        return 1 if x ** 2 + y ** 2 < 1 else 0

    count = sc.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
    print("Pi is roughly %f" % (4.0 * count / n))

    sc.stop()


"""

In [ ]:
!spark-2.0.1-bin-hadoop2.7/./bin/spark-submit /home/cc/spark-2.0.1-bin-hadoop2.7/pi.py 10

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.5em;"> **Example: Wordcount.py:** </span>

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;"> Using all the concepts of RDDs we learned in the previous lesson, we will move to implement an example of word count for an existing file inside the spark folder we have called as "CHANGES.txt" </span>

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;"> The main purpose of this progrmam is to calculate the wordcount of the given file, in this case it is text. This would be a good usecase when we are dealing with some of the big chunk file & need some set of things to be calculated. </span>


In [ ]:
#!cd /home/cc/bin
#!pwd
#!ls -la

In [ ]:
"""
# NonExecutable file
"""

from __future__ import print_function

import sys
from operator import add

from pyspark import SparkContext


if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: wordcount <file>", file=sys.stderr)
        exit(-1)
    sc = SparkContext(appName="PythonWordCount")
    lines = sc.textFile(sys.argv[1], 1)
    counts = lines.flatMap(lambda x: x.split(' ')) \
                  .map(lambda x: (x, 1)) \
                  .reduceByKey(add)
    output = counts.collect()
    for (word, count) in output:
        print("%s: %i" % (word, count))

    sc.stop()
"""
"""

In [ ]:
!spark-2.0.1-bin-hadoop2.7/./bin/spark-submit /home/cc/spark-2.0.1-bin-hadoop2.7/wordcount.py spark-2.0.1-bin-hadoop2.7/CHANGES.txt

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.5em;"> **Example(Big Data): Airline Delay** </span>

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;"> In order to demonstrate a common use of Spark, let's take a look at a common use case where we read in a CSV file of data and compute some aggregate statistic. In this case, we're looking at the on-time flight data set from the U.S. Department of Transportation, recording all U.S. domestic flight departure and arrival times along with their departure and arrival delays for the month of April, 2014. I typically use this data set because one month is manageable for exploration, but the entire data set needs to be computed upon with a cluster. The entire app is as follows: </span>

In [ ]:
"""
## Spark Application - execute with spark-submit
"""

## Imports
import csv
import matplotlib.pyplot as plt

from StringIO import StringIO
from datetime import datetime
from collections import namedtuple
from operator import add, itemgetter
from pyspark import SparkConf, SparkContext

## Module Constants
APP_NAME = "Flight Delay Analysis"
DATE_FMT = "%Y-%m-%d"
TIME_FMT = "%H%M"

fields   = ('date', 'airline', 'flightnum', 'origin', 'dest', 'dep',
            'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')
Flight   = namedtuple('Flight', fields)

## Closure Functions
def parse(row):
    """
    Parses a row and returns a named tuple.
    """

    row[0]  = datetime.strptime(row[0], DATE_FMT).date()
    row[5]  = datetime.strptime(row[5], TIME_FMT).time()
    row[6]  = float(row[6])
    row[7]  = datetime.strptime(row[7], TIME_FMT).time()
    row[8]  = float(row[8])
    row[9]  = float(row[9])
    row[10] = float(row[10])
    return Flight(*row[:11])

def split(line):
    """
    Operator function for splitting a line with csv module
    """
    reader = csv.reader(StringIO(line))
    return reader.next()

def plot(delays):
    """
    Show a bar chart of the total delay per airline
    """
    airlines = [d[0] for d in delays]
    minutes  = [d[1] for d in delays]
    index    = list(xrange(len(airlines)))

    fig, axe = plt.subplots()
    bars = axe.barh(index, minutes)

    # Add the total minutes to the right
    for idx, air, min in zip(index, airlines, minutes):
        if min > 0:
            bars[idx].set_color('#d9230f')
            axe.annotate(" %0.0f min" % min, xy=(min+1, idx+0.5), va='center')
        else:
            bars[idx].set_color('#469408')
            axe.annotate(" %0.0f min" % min, xy=(10, idx+0.5), va='center')

    # Set the ticks
    ticks = plt.yticks([idx+ 0.5 for idx in index], airlines)
    xt = plt.xticks()[0]
    plt.xticks(xt, [' '] * len(xt))

    # minimize chart junk
    plt.grid(axis = 'x', color ='white', linestyle='-')

    plt.title('Total Minutes Delayed per Airline')
    save_name = 'home/cc/spark-2.0.1-bin-hadoop2.7/image.png'
    plt.savefig(save_name)
    plt.show()
    plt.close()

## Main functionality
def main(sc):

    # Load the airlines lookup dictionary
    airlines = dict(sc.textFile("airlines.csv").map(split).collect())

    # Broadcast the lookup dictionary to the cluster
    airline_lookup = sc.broadcast(airlines)

    # Read the CSV Data into an RDD
    flights = sc.textFile("flights.csv").map(split).map(parse)

    # Map the total delay to the airline (joined using the broadcast value)
    delays  = flights.map(lambda f: (airline_lookup.value[f.airline],
                                     add(f.dep_delay, f.arv_delay)))

    # Reduce the total delay for the month to the airline
    delays  = delays.reduceByKey(add).collect()
    delays  = sorted(delays, key=itemgetter(1))

    # Provide output from the driver
    for d in delays:
        print "%0.0f minutes delayed\t%s" % (d[1], d[0])

    # Show a bar chart of the delays
    plot(delays)

if __name__ == "__main__":
    # Configure Spark
    conf = SparkConf().setMaster("local[*]")
    conf = conf.setAppName(APP_NAME)
    sc   = SparkContext(conf=conf)

    # Execute Main functionality
    main(sc)

""""""
"""

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;"> So what is this code doing?
Let's look particularly at the main function which does the work most directly related to Spark. First, we load up a **CSV file** into an RDD, then map the split function to it. The split function parses each line of text using the csv module and returns a tuple that represents the row. Finally we pass the collect action to the RDD, which brings the data from the RDD back to the driver as a Python list. In this case, **airlines.csv** is a small jump table that will allow us to join airline codes with the airline full name. We will store this jump table as a Python dictionary and then broadcast it to the node  sc.broadcast, which also works for the case when we have a cluster of Spark. 

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;"> Next, the main function loads the much larger **flights.csv**. After splitting the CSV rows, we map the parse function to the CSV row, which converts dates and times to Python dates and times, and casts floating point numbers appropriately. It also stores the row as a NamedTuple called Flight for efficient ease of use. </span>
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;">With an RDD of Flight objects in hand, we map an anonymous function that transforms the RDD to a series of key-value pairs where the key is the name of the airline and the value is the sum of the arrival and departure delays. Each airline has its delay summed together using the reduceByKey action and the add operator, and this RDD is collected back to the driver (again the number airlines in the data is relatively small). Finally the delays are sorted in ascending order, then the output is printed to the console as well as visualized using matplotlib. </span>

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;"> To run this code, use the spark-submit command as follows:</span>

In [1]:
!spark-2.0.1-bin-hadoop2.7/./bin/spark-submit /home/cc/spark-2.0.1-bin-hadoop2.7/flightdelay.py

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
16/11/06 06:54:51 INFO SparkContext: Running Spark version 2.0.1
16/11/06 06:54:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/11/06 06:54:51 WARN Utils: Your hostname, nimish-spark2 resolves to a loopback address: 127.0.0.1; using 192.168.0.206 instead (on interface eth0)
16/11/06 06:54:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
16/11/06 06:54:51 INFO SecurityManager: Changing view acls to: root
16/11/06 06:54:51 INFO SecurityManager: Changing modify acls to: root
16/11/06 06:54:51 INFO SecurityManager: Changing view acls groups to: 
16/11/06 06:54:51 INFO SecurityManager: Changing modify acls groups to: 
16/11/06 06:54:51 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(root); groups with view permissions: Set(); users  with mo

<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;"> This will generathe the below figure: </span>
<div style="width:830; background-color:white; height:540px; overflow:scroll; overflow-x: scroll;overflow-y: hidden;">

<img style=" float:left; display:inline" src="" width="160" height="70"/>

<img style=" float:left; display:inline" src="http://129.114.111.241:8888/tree/spark-2.0.1-bin-hadoop2.7/delays.png" width="860" height="420"/> </div>


<span style="color:#000; font-family: 'Bebas Neue'; font-size: 1.3em;">This will create a Spark job using the localhost as the master, and look for the two **CSV** files in an ontime directory that is in the same directory as the python code is in. The final result shows that the total delays (in minutes) for the perticlar month. on our case we took the data for the month of April that goes from arriving early if you're flying out of the continental U.S. to Hawaii or Alaska to an aggregate total delay for most big airlines. Note especially that we can visualize the result using matplotlib directly on the driver program, with the above python code: </span>